In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
response = requests.get("https://www.pukanina.com/faq-foire-aux-questions-infos-pratiques/")
QandA = BeautifulSoup(response.text).select(".et_pb_toggle")

questions = [elem.h5.text for elem in QandA]
answers = [elem.div.text[1:] for elem in QandA]

In [3]:
df = pd.DataFrame(data = {'Questions': questions, 'Answers': answers})
df.head(3)

,Questions,Answers
0,Pour et contre d'un voyage groupé en tout peti...,Nous programmons des voyages groupés à dates f...
1,Pour et contre d'un voyage sur mesure/individu...,Avantages:\n\nun circuit individuel est modifi...
2,Pourquoi vous n'incluez pas les commissions ba...,Nous n’incluons pas les commissions bancaires ...


In [4]:
df.to_csv('Q&A.csv', sep = ';', index = False, encoding = 'utf-8', na_rep = 'NA')